# Naive Shapeshifting Algorithm

In this Notebook we will implement a naive solution for solving the shapeshifting problem. We do this by using one coordinator as the root of the new implementation and the other coordinator boat to move workers to different locations on the new configuration

## Methodology

To solve this problem we will take advantage of the geometries of the boats and how they are connected to find
the maximum overlap between our current configuration and our goal configuration at each step.


In [2]:
import graph_tool.all as gt

/usr/local/Cellar/graph-tool/2.27_2/lib/python3.7/site-packages/graph_tool/draw/cairo_draw.py:1494: RuntimeWarning: Error importing Gtk module: No module named 'gi'; GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)


In [25]:
# This function will return a list of latch and unlatch steps in order to transform the initial configuration
# to the goal configuration
def shapeshift(initial_configuration, goal_configuration):
    current_goal_config = None
    current_start_config = initial_configuration
    planner_steps = []
    while current_goal_config != goal_configuration:
        if current_goal_config == None:
            goal = goal_configuration
        else:
            goal = goal_configuration - current_goal_config
        current_steps = find_overlap(current_start_config, goal)
        planner_steps.append(current_steps)
        break
    return planner_steps
        
def find_overlap(current_configuration, goal_configuration):
    current_children = current_configuration.new_vertex_property("vector<int>")
    goal_children = goal_configuration.new_vertex_property("vector<int>")
    
    current_parent = current_configuration.new_vertex_property("int")
    goal_parent = goal_configuration.new_vertex_property("int")
    
    gt.bfs_search(current_configuration, current_configuration.vertex(0), 
               VisitorDistance("current", current_parent, current_children))
    gt.bfs_search(goal_configuration, goal_configuration.vertex(0), 
               VisitorDistance("goal", goal_parent, goal_children))
    
#     current_index_dist = [(i,current_dist.a[i]) for i in range(len(current_dist.a))]
#     goal_index_dist = [(i,goal_dist.a[i]) for i in range(len(goal_dist.a))]
    
#     current_index_dist.sort(key = lambda x : x[1])
#     goal_index_dist.sort(key = lambda x : x[1])
    
    matching = []
    for potential_leaf in current_configuration.vertices():
        if len(current_children[potential_leaf]) > 0 or potential_leaf == 0:
            continue
        for potential_root in goal_configuration.vertices():
            if goal_parent[potential_root] != 0 or potential_root == 0:
                continue
            stack_current = [potential_leaf]
            stack_goal = [potential_root]
            current_overlap = []
            while len(stack_current) != 0:
                current_node = stack_current.pop()
                current_goal = stack_goal.pop()
                current_overlap.append((current_node, current_goal))
                
#                 if (current_configuration.edge_properties["connection_type"][current_node.incoming_edge()]
#                 not in [edge_propercurrent_goal.edge_list())]:
#                     break
                for edge in current_goal.out_edges():
                    if current_configuration.edge(current_node, current_parent[current_node]) == None:
                        continue
                    if (goal_configuration.edge_properties["connection_type"][edge] == 
                    current_configuration.edge_properties["connection_type"][current_configuration.edge(current_node, current_parent[current_node])]):
                        stack_current.append(current_parent[current_node])
                        stack_goal.append(edge.target())
            if len(current_overlap) > len(matching):
                        matching = current_overlap
    return matching

class VisitorDistance(gt.BFSVisitor):

    def __init__(self, name, parent, children):
        self.name = name
        self.parent = parent
        self.children = children

    def tree_edge(self, e):
        self.parent[e.target()] = e.source()
        self.children[e.source()].append(e.target())
    

# Examples

## Identity Example - Initial and Goal Configurations are the same

In [4]:
# Make initial configuration
init_config = gt.Graph(directed = False)
coordinator = init_config.add_vertex()
side_boat = init_config.add_vertex()
latch = init_config.add_edge(coordinator, side_boat)
edge_types = init_config.new_edge_property("int")
edge_types[latch] = 1

In [5]:
# Make goal configuration, it's the same
goal_config = gt.Graph(directed = False)
goal_coord = goal_config.add_vertex()
goal_side = goal_config.add_vertex()
goal_latch = goal_config.add_edge(goal_coord, goal_side)
goal_edge_types = goal_config.new_edge_property("int")
goal_edge_types[goal_latch] = 1

In [5]:
latching_plan = shapeshift(init_config, goal_config)
print(latching_plan)

0
0
[[]]


## Example from Thesis proposal

In [26]:
init_config = gt.Graph(directed = False)
# initialize boats
init_coord = init_config.add_vertex()
init_work1 = init_config.add_vertex()
init_work2 = init_config.add_vertex()
init_work3 = init_config.add_vertex()
init_work4 = init_config.add_vertex()
init_work5 = init_config.add_vertex()

# initialize edges
i_coord_work1 = init_config.add_edge(init_coord, init_work1)
i_coord_work2 = init_config.add_edge(init_coord, init_work2)
i_work1_work3 = init_config.add_edge(init_work1, init_work3)
i_work2_work3 = init_config.add_edge(init_work2, init_work3)
i_work2_work4 = init_config.add_edge(init_work2, init_work4)
i_work3_work5 = init_config.add_edge(init_work3, init_work5)
i_work4_work5 = init_config.add_edge(init_work4, init_work5)

# label edges
init_edge_types = init_config.new_edge_property("int")
init_edge_types[i_coord_work1] = 1
init_edge_types[i_coord_work2] = 2
init_edge_types[i_work1_work3] = 2
init_edge_types[i_work2_work3] = 1
init_edge_types[i_work2_work4] = 2
init_edge_types[i_work3_work5] = 2
init_edge_types[i_work4_work5] = 1
init_config.edge_properties['connection_type'] = init_edge_types

In [27]:
goal_config = gt.Graph(directed = False)
#initialize boats
goal_coord = goal_config.add_vertex()
goal_work1 = goal_config.add_vertex()
goal_work2 = goal_config.add_vertex()
goal_work3 = goal_config.add_vertex()
goal_work4 = goal_config.add_vertex()
goal_work5 = goal_config.add_vertex()

#initialize edges
g_work1_work2 = goal_config.add_edge(goal_work1, goal_work2)
g_work1_work4 = goal_config.add_edge(goal_work1, goal_work4)
g_work2_work3 = goal_config.add_edge(goal_work2, goal_work3)
g_work2_work5 = goal_config.add_edge(goal_work2, goal_work5)
g_work3_coord = goal_config.add_edge(goal_work3, goal_coord)
g_work4_work5 = goal_config.add_edge(goal_work4, goal_work5)
g_work5_coord = goal_config.add_edge(goal_work5, goal_coord)

#label edges
goal_edge_types = goal_config.new_edge_property("int")
goal_edge_types[g_work1_work2] = 1
goal_edge_types[g_work1_work4] = 2
goal_edge_types[g_work2_work3] = 1
goal_edge_types[g_work2_work5] = 2
goal_edge_types[g_work3_coord] = 2
goal_edge_types[g_work4_work5] = 1
goal_edge_types[g_work5_coord] = 1
goal_config.edge_properties['connection_type'] = goal_edge_types

In [28]:
latching_plan = shapeshift(init_config, goal_config)
print(latching_plan)

leafs
4
roots
3
roots
5
leafs
5
roots
3
roots
5
[[(<Vertex object with index '5' at 0x130230810>, <Vertex object with index '5' at 0x1302305d0>), (3, <Vertex object with index '2' at 0x130230030>), (1, <Vertex object with index '5' at 0x130230690>), (0, <Vertex object with index '4' at 0x130230930>), (0, <Vertex object with index '0' at 0x1302306f0>)]]
